In [21]:
import os
import re
import pickle
import numpy as np

from PIL import Image

%matplotlib inline

from matplotlib import pyplot as plt

masks_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/masks/event/segmentation_masks'

events_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/event/NIR_SWIR'

train_dataset = (os.path.join(os.getcwd(),'model_training','train_dataset'))
os.makedirs(train_dataset,exist_ok=True)

train_images = (os.path.join(os.getcwd(),'train_dataset','train','images'))
train_masks = (os.path.join(os.getcwd(),'train_dataset','train','masks'))

val_images = (os.path.join(os.getcwd(),'train_dataset','val','images'))
val_masks = (os.path.join(os.getcwd(),'train_dataset','val','masks'))

test_images = (os.path.join(os.getcwd(),'train_dataset','test','images'))
test_masks = (os.path.join(os.getcwd(),'train_dataset','test','masks'))


os.makedirs(train_images,exist_ok=True)
os.makedirs(train_masks,exist_ok=True)
os.makedirs(val_images,exist_ok=True)
os.makedirs(val_masks,exist_ok=True)
os.makedirs(test_images,exist_ok=True)
os.makedirs(test_masks,exist_ok=True)

train_max_idx = np.round(len(os.listdir(masks_path))*0.8).astype(int)

val_max_idx = train_max_idx + np.round(len(os.listdir(masks_path))*0.1).astype(int)

test_max_idx = len(os.listdir(masks_path))

# train_test = []
# val_test = []
# test_test = []


for idx,mask_name in enumerate(os.listdir(masks_path)):
    patch_name = re.match(r'(.+)_mask',mask_name).group(1)
    patch_name = mask_name.replace('mask_weakly','NIR_SWIR')
    
    with open(os.path.join(masks_path,mask_name),'rb') as mask_file:
        mask = pickle.load(mask_file)

    mask[mask == 122] = 0
    # mask[mask == 255] = 1

    mask_img = Image.fromarray(mask)

    
    # print(type(mask),mask.dtype,mask.shape)
    # break

    with open(os.path.join(events_path,patch_name),'rb') as image_file:
        image = pickle.load(image_file)
        image_uint8 = np.round(np.multiply(image,255)).astype(np.uint8)

        image_img = Image.fromarray(image_uint8)

    # plt.figure()
    # plt.imshow(image)
    # plt.figure()
    # plt.imshow(image_uint8)
    # image_img.show()

    # break

    if idx<= train_max_idx:
        # train_test.append('a')

        mask_img.save(os.path.join(train_masks,mask_name.replace('_mask_weakly.pkl','_mask.png')))
        image_img.save(os.path.join(train_images,patch_name.replace('.pkl','.png')))
        
        
        # with open(os.path.join(train_masks,mask_name.replace('_mask_weakly','_mask')),'wb') as mask_file:
        #     pickle.dump(mask,mask_file)
        # with open(os.path.join(train_images,patch_name),'wb') as image_file:
        #     pickle.dump(image,image_file)

    elif (idx> train_max_idx) and (idx<= val_max_idx) :
        # val_test.append('a')

        mask_img.save(os.path.join(val_masks,mask_name.replace('_mask_weakly.pkl','_mask.png')))
        image_img.save(os.path.join(val_images,patch_name.replace('.pkl','.png')))

        # with open(os.path.join(val_masks,mask_name.replace('_mask_weakly','_mask')),'wb') as mask_file:
        #     pickle.dump(mask,mask_file)
        # with open(os.path.join(val_images,patch_name),'wb') as image_file:
        #     pickle.dump(image,image_file)
    
    elif (idx> val_max_idx) and (idx<= test_max_idx):
        # test_test.append('a')
        mask_img.save(os.path.join(test_masks,mask_name.replace('_mask_weakly.pkl','_mask.png')))
        image_img.save(os.path.join(test_images,patch_name.replace('.pkl','.png')))

        # with open(os.path.join(test_masks,mask_name.replace('_mask_weakly','_mask')),'wb') as mask_file:
        #     pickle.dump(mask,mask_file)
        # with open(os.path.join(test_images,patch_name),'wb') as image_file:
        #     pickle.dump(image,image_file)

# print(len(train_test),len(val_test),len(test_test))
    # print(os.path.isfile(os.path.join(events_path,patch_name)))

    # break



In [5]:
import os
import re
import pickle
import numpy as np


from PIL import Image


masks_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/masks/event/segmentation_masks'

# events_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/event/NIR_SWIR'
notevents_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/notevent/NIR_SWIR'


train_images = (os.path.join(os.getcwd(),'train_dataset','train','images'))
train_masks = (os.path.join(os.getcwd(),'train_dataset','train','masks'))

val_images = (os.path.join(os.getcwd(),'train_dataset','val','images'))
val_masks = (os.path.join(os.getcwd(),'train_dataset','val','masks'))

test_images = (os.path.join(os.getcwd(),'train_dataset','test','images'))
test_masks = (os.path.join(os.getcwd(),'train_dataset','test','masks'))

test_max_cont_end_loop = len(os.listdir(masks_path)) # Last index of where the data will be gathered
train_max_cont_end_loop = np.round(test_max_cont_end_loop*0.8).astype(int)

val_max_cont_end_loop = train_max_cont_end_loop + np.round(test_max_cont_end_loop*0.1).astype(int)


mask = Image.fromarray(np.zeros((256,256,3),dtype=np.uint8))


with open('/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/granules_completed.txt','r') as file:
    n_granules = len(file.readlines())

n_images_per_granule = np.round(test_max_cont_end_loop/n_granules)



train_test = []
val_test = []
test_test = []

cont = 0
cont_end_loop = 1
scene_names_list = []
null_image = []
for image_name in os.listdir(notevents_path):
    
    
    scene_name = re.match(r'(.+)_G',image_name).group(1)

    # if (scene_name not in scene_names_list):
    if scene_names_list.count(scene_name)<6:
        
            
        scene_names_list.append(scene_name)

        # print(cont, cont_end_loop,scene_name)
        # cont +=1
        
        with open(os.path.join(notevents_path,image_name),'rb') as image_file:
            image = pickle.load(image_file)

            image_uint8 = np.round(np.multiply(image,255)).astype(np.uint8)
            image_img = Image.fromarray(image_uint8)
        # image_img.show()
        # break
        if cont_end_loop<= train_max_cont_end_loop:
            
            train_test.append(scene_name)
            
            mask.save(os.path.join(train_masks,image_name.replace('_NIR_SWIR.pkl','_mask.png')))
            image_img.save(os.path.join(train_images,image_name.replace('.pkl','.png')))

            cont_end_loop +=1

            # with open(os.path.join(train_masks,image_name.replace('_NIR_SWIR','_mask')),'wb') as mask_file:
            #     pickle.dump(mask,mask_file)
            # with open(os.path.join(train_images,image_name),'wb') as image_file:
            #     pickle.dump(image,image_file)

        elif (cont_end_loop> train_max_cont_end_loop) and (cont_end_loop<= val_max_cont_end_loop) :

            val_test.append(scene_name)
            mask.save(os.path.join(val_masks,image_name.replace('_NIR_SWIR.pkl','_mask.png')))
            image_img.save(os.path.join(val_images,image_name.replace('.pkl','.png')))
            
            cont_end_loop +=1

            # with open(os.path.join(val_masks,image_name.replace('_NIR_SWIR','_mask')),'wb') as mask_file:
            #     pickle.dump(mask,mask_file)
            # with open(os.path.join(val_images,image_name),'wb') as image_file:
            #     pickle.dump(image,image_file)
        
        elif (cont_end_loop> val_max_cont_end_loop) and (cont_end_loop<= test_max_cont_end_loop):
            
            test_test.append(scene_name)

            mask.save(os.path.join(test_masks,image_name.replace('_NIR_SWIR.pkl','_mask.png')))
            image_img.save(os.path.join(test_images,image_name.replace('.pkl','.png')))

            cont_end_loop +=1

            # with open(os.path.join(test_masks,image_name.replace('_NIR_SWIR','_mask')),'wb') as mask_file:
            #     pickle.dump(mask,mask_file)
            # with open(os.path.join(test_images,image_name),'wb') as image_file:
            #     pickle.dump(image,image_file)

        else:
            # print(cont_end_loop)
            break
        # print(image_name)
        # print(os.path.isfile(os.path.join(events_path,image_name)))



In [35]:
import os
import re
import pickle
import numpy as np

import shutil
from PIL import Image
from pathlib import Path

masks_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/masks/event/segmentation_masks'

events_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/event/NIR_SWIR'
notevents_main_folder = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/notevent'




def potsprocess_null_patches(patches_main_folder):

    null_patches_path = os.path.join(patches_main_folder,'Null patches')
    os.makedirs(null_patches_path,exist_ok=True)

    folders_list = ['NIR_SWIR','RGB','Vegetation Red','NIR1']

    for dir in folders_list:
        os.makedirs(os.path.join(null_patches_path,dir),exist_ok=True)

    Path(os.path.join(null_patches_path,'null_events_list.txt')).touch(exist_ok=True)

    already_classified_condition = False

    patches_removed = []

    for notevent_folder in folders_list:
        notevent_folder_path = os.path.join(patches_main_folder,notevent_folder)
        for notevent_name in os.listdir(notevent_folder_path):
            notevent_path = os.path.join(notevent_folder_path,notevent_name)
            image_name = os.path.basename(notevent_path)

            with open(notevent_path,'rb') as image_file:
                image = pickle.load(image_file)
            
            if (image.max(),image.min()) == (0,0):
                with open(os.path.join(null_patches_path,'null_events_list.txt'),'r') as f:
                    for classified_name in f.readlines():
                        if image_name == classified_name.rstrip('\n'):
                            already_classified_condition =True
                            break
                if not already_classified_condition:

                    with open(os.path.join(null_patches_path,'null_events_list.txt'),'a') as txt_file:
                        txt_file.write(f'{image_name}\n')
                    
                    folder_name = re.findall(r'\)_([^\.]+)\.pkl', image_name)[0]
                    if image_name.replace(f'_{folder_name}.pkl','') not in patches_removed:
                        for folder in folders_list:
                            folder_id = folder
                            if folder == 'Vegetation Red':
                                folder_id = 'VEG'

                            image_folder_name = image_name.replace(folder_name,folder_id)
                            shutil.copyfile(os.path.join(os.path.dirname(notevent_folder_path),folder,image_folder_name),os.path.join(null_patches_path,folder,image_folder_name))
                            
                            os.remove(os.path.join(patches_main_folder,folder,image_folder_name)) # Delete the file from its original location

                        patches_removed.append(image_name.replace(f'_{folder_name}.pkl',''))
    
    print(f'{len(patches_removed)} null patches have been reclassified. ')

# potsprocess_null_patches(notevents_main_folder)


2112 null patches have been reclassified. 


In [8]:
with open('patches_to_revise.txt','w') as f:
    for elem in null_notevents:
        f.write(f'{elem}\n')

In [ ]:
import os
import re
import pickle
import numpy as np

from PIL import Image

%matplotlib inline

import matplotlib as mpl

from matplotlib import pyplot as plt

masks_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/masks/event/segmentation_masks'

events_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/event/NIR_SWIR'

# dataset_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset'


dataset_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(__file__))),'dataset')

masks_path_2 = os.path.join(dataset_path,'masks','weakly_segmentation')

events_path_2 = os.path.join(dataset_path,'images','event','NIR_SWIR')


train_dataset = (os.path.join(os.path.dirname(dataset_path),'model_training','train_dataset'))
os.makedirs(train_dataset,exist_ok=True)

train_images = (os.path.join(os.path.dirname(dataset_path),'train_dataset','train','images'))
train_masks = (os.path.join(os.path.dirname(dataset_path),'train_dataset','train','masks'))

val_images = (os.path.join(os.path.dirname(dataset_path),'train_dataset','val','images'))
val_masks = (os.path.join(os.path.dirname(dataset_path),'train_dataset','val','masks'))

test_images = (os.path.join(os.path.dirname(dataset_path),'train_dataset','test','images'))
test_masks = (os.path.join(os.path.dirname(dataset_path),'train_dataset','test','masks'))


os.makedirs(train_images,exist_ok=True)
os.makedirs(train_masks,exist_ok=True)
os.makedirs(val_images,exist_ok=True)
os.makedirs(val_masks,exist_ok=True)
os.makedirs(test_images,exist_ok=True)
os.makedirs(test_masks,exist_ok=True)

train_split_ratio : float = 0.8
val_split_ratio : float = 0.1
test_split_ratio : float = 0.1

train_max_idx = np.round(len(os.listdir(masks_path))*0.8).astype(int)

val_max_idx = train_max_idx + np.round(len(os.listdir(masks_path))*0.1).astype(int)

test_max_idx = len(os.listdir(masks_path))


weakly = False

for idx,mask_name in enumerate(os.listdir(masks_path)):
    patch_name = re.match(r'(.+)_mask',mask_name).group(1)
    patch_name = mask_name.replace('mask_weakly','NIR_SWIR')
    
    with open(os.path.join(masks_path,mask_name),'rb') as mask_file:
        mask = pickle.load(mask_file)

    if not weakly: #For supervised binary segmentation
        mask[mask == -1] = 0


    with open(os.path.join(events_path,patch_name),'rb') as image_file:
        image = pickle.load(image_file)


    if idx<= train_max_idx:

        mask_img.save(os.path.join(train_masks,mask_name.replace('_mask_weakly.pkl','_mask.png')))
        image_img.save(os.path.join(train_images,patch_name.replace('.pkl','.png')))
        

    elif (idx> train_max_idx) and (idx<= val_max_idx) :

        mpl.image.imsave(os.path.join(val_masks,mask_name.replace('_mask_weakly.pkl','_mask.png')),mask)
        mpl.image.imsave(os.path.join(val_images,patch_name.replace('.pkl','.png')),image)

    elif (idx> val_max_idx) and (idx<= test_max_idx):

        mpl.image.imsave(os.path.join(test_masks,mask_name.replace('_mask_weakly.pkl','_mask.png')),mask)
        mpl.image.imsave(os.path.join(test_images,patch_name.replace('.pkl','.png')),image)



In [4]:
import os
import re
import pickle
import numpy as np


from PIL import Image


masks_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/masks/event/segmentation_masks'

# events_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/event/NIR_SWIR'
notevents_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/notevent/NIR_SWIR'


test_max_cont_end_loop = len(os.listdir(masks_path)) # Last index of where the data will be gathered
train_max_cont_end_loop = np.round(test_max_cont_end_loop*test_split_ratio).astype(int)

val_max_cont_end_loop = train_max_cont_end_loop + np.round(test_max_cont_end_loop*val_split_ratio).astype(int)



mask = np.zeros((256,256,1),dtype=np.float32)

# n_granules_txt = os.path.join(dataset_path,'granules_completed.txt')

with open(os.path.join(dataset_path,'granules_completed.txt'),'r') as file:
    n_granules = len(file.readlines())

n_images_per_granule = np.round(test_max_cont_end_loop/n_granules)

cont_end_loop = 1
scene_names_list = []
for image_name in os.listdir(notevents_path):
        
    scene_name = re.match(r'(.+)_G',image_name).group(1)

    if scene_names_list.count(scene_name)<n_images_per_granule:
                    
        scene_names_list.append(scene_name)

        with open(os.path.join(notevents_path,image_name),'rb') as image_file:
            image = pickle.load(image_file)

        if cont_end_loop<= train_max_cont_end_loop:

            
            mpl.image.imsave(os.path.join(train_dataset,'train','masks',mask_name.replace('_mask_weakly.pkl','_mask.png')),mask)
            mpl.image.imsave(os.path.join(train_dataset,'train','images',patch_name.replace('.pkl','.png')),image)
            

            cont_end_loop +=1

        elif (cont_end_loop> train_max_cont_end_loop) and (cont_end_loop<= val_max_cont_end_loop) :

            mpl.image.imsave(os.path.join(train_dataset,'val','masks',mask_name.replace('_mask_weakly.pkl','_mask.png')),mask)
            mpl.image.imsave(os.path.join(train_dataset,'val','images',patch_name.replace('.pkl','.png')),image)
            
            cont_end_loop +=1

        
        elif (cont_end_loop> val_max_cont_end_loop) and (cont_end_loop<= test_max_cont_end_loop):

            mpl.image.imsave(os.path.join(train_dataset,'test','masks',mask_name.replace('_mask_weakly.pkl','_mask.png')),mask)
            mpl.image.imsave(os.path.join(train_dataset,'test','images',patch_name.replace('.pkl','.png')),image)

            cont_end_loop +=1

        else:
            print('Reached maximum number of not events for the dataset')
            break




a
